In [1]:
import base64

import requests
import json

In [2]:
# CLIENT NEEDS TO DO THIS
with open('1.jpg', 'rb') as f:
    im_bytes = f.read()
im_b64 = base64.b64encode(im_bytes).decode('utf-8')

In [3]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
payload = json.dumps({"image": im_b64})

r = requests.post('http://localhost:8000/infer', data=payload, headers=headers)
res = json.loads(r.text)

In [4]:
def dim(a):
    if not type(a) == list:
        return []
    return [len(a)] + dim(a[0])

In [5]:
dim(res['embs'])

[1, 3, 2048]

### FOR DEBUGGING

In [ ]:
def read_yaml(file_path='./fast_api/src/config.yaml'):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

config = read_yaml()

import numpy as np
import yaml
import random
import cv2
from imageio.v2 import imread

import sys
import torch
sys.path.append("./fast_api/src/yolo")

from utils.datasets import letterbox
from models.experimental import attempt_load
from utils.torch_utils import select_device, load_classifier
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, plot_one_box, strip_optimizer, get_embedding)

config['weights']='fast_api/src/yolov4-p5_.pt'

In [ ]:
device = '0'
device = select_device(device)
half = device.type != 'cpu'
model = attempt_load(config['weights'], map_location=device) # Load YOLO Model
imgsz = check_img_size(config['img_size'], s=model.stride.max())  # check img_size
if half:
    model.half()  # to FP16
if config['emb']:
    # Load pre-saved resnet model locally, so don't need to download each time container spins up
    classify_model = load_classifier(name='resnet101', n=2, local='fast_api/src/resnet101.pt') 
    classify_model.to(device).eval()
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

In [ ]:
img_bytes = base64.b64decode(im_b64.encode('utf-8'))
img = imread(img_bytes)
# img0 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
img0 = img

In [ ]:
img = letterbox(img0, new_shape=640)[0]

img = img[:, :, ::-1].transpose(2, 0, 1)

img = np.ascontiguousarray(img)

img = torch.from_numpy(img).to(device)
img = img.half() if half else img.float()
img /= 255.0
if img.ndimension() == 3:
    img = img.unsqueeze(0)

In [ ]:
pred = model(img, augment=config['augment'])[0]
pred = non_max_suppression(pred, config['conf_thres'], config['iou_thres'], classes=None, agnostic=False)


In [ ]:
for i, det in enumerate(pred):

    s, im0 = '', img0
    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
    if det is not None and len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
        print(det)
        
        for *xyxy, conf, cls in det:
            label = '%s' % (names[int(cls)])
            print(label)
            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=2)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(im0, interpolation='nearest')
plt.imsave('test.png', im0)